### Задание 1: составить словари для классификации по тональности.

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    - тест на определение ближайших слов
    - тест на аналогии (мужчина – король : женщина – королева)
    - тест на определение лишнего слова.
4. Постройте несколько визуализаций:
    - TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    - задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [1]:
import json
import bz2
import regex
from scipy import sparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tqdm import tqdm
tqdm.pandas()

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from collections import Counter
from itertools import chain
from pymorphy3 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
from gensim.models.word2vec import Word2Vec
import regex as re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\garti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\garti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [00:41, 4884.33it/s]


In [4]:
df = pd.DataFrame(responses)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153499 entries, 0 to 153498
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city                138325 non-null  object 
 1   rating_not_checked  153499 non-null  bool   
 2   title               153499 non-null  object 
 3   num_comments        153499 non-null  int64  
 4   bank_license        153498 non-null  object 
 5   author              153479 non-null  object 
 6   bank_name           153499 non-null  object 
 7   datetime            153499 non-null  object 
 8   text                153499 non-null  object 
 9   rating_grade        88658 non-null   float64
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 10.7+ MB


In [6]:
# функции предобработки текста

# формируем словарь стоп-слов
mystopwords = stopwords.words('russian') + [
              'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
              'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой', 'руб', 'г']

# убераем пунктуацию    
def words_only(text):
    return " ".join(re.compile("[А-Яа-яЁё]+").findall(text))

# приводим слова к нормальной форме
def lemmatize(text, m=MorphAnalyzer()):
    try:
        return [m.parse(word)[0].normal_form for word in text.split(' ')]
    except:
        pass

# исключаем стоп-слова    
def remove_stopwords(text, stopwords=mystopwords):
    try:
        return " ".join([token for token in text if not token in stopwords])
    except:
        pass
    
# основная функция
def preprocess(text):
    text = nltk.tokenize.sent_tokenize(text) #делим на предложения
    return np.array([remove_stopwords(lemmatize(words_only(sentence.lower()))) for sentence in text]) # сборка всех функций 

In [7]:
df['text_pp'] = df['text'].progress_apply(preprocess)

100%|████████████████████████████████████████████████████████████████████████| 153499/153499 [1:56:23<00:00, 21.98it/s]


In [8]:
# Сохраняем DataFrame в файл CSV
df.to_csv('df_pp.csv', index=False)

In [ ]:
# Загрузка DataFrame
df = pd.read_csv('df_pp.csv')

In [36]:
# Инициаоищация модели
# 
model = Word2Vec()
model.build_vocab(tqdm(sentence for text in df['text_pp'] for sentence in text))

2664065it [00:27, 97592.58it/s] 

CPU times: total: 27.3 s
Wall time: 27.3 s


In [34]:
%%time
model.train(other_sentences)
corpus = (sentence for text in df['text_pp'] for sentence in text)
model = Word2Vec(corpus, workers=4, vector_size=300, min_count=1, window=7, epochs=15)

TypeError: Using a generator as corpus_iterable can't support 16 passes. Try a re-iterable sequence.

In [32]:
next(corpus)

'добрый день'

In [ ]:
temp = df['text_pp'][:2].values
np.hstack(temp).shape

In [ ]:
temp.shape

In [ ]:
%%time
model = Word2Vec(df['text_pp'], workers=4, vector_size=300, min_count=1, window=7, epochs=15)

In [ ]:
# тест на определение ближайших слов  #most_similar('сотрудник')
for i, w in enumerate(model.wv.index_to_key):
    if i == 100:
        break
    print(i, w)

In [ ]:
model.wv.most_similar('телефон')

In [ ]:
# тест на аналогию
model.wv.most_similar(positive=['отделение банк'], negative=['офис'])

In [ ]:
# тест на определение лишнего слова
model.wv.doesnt_match(['кредит', 'повод', 'росспотребнадзор'])

### Часть 2. Распространение метки
Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8 негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
1. `graphlab.label_propagation` (`graphlab` доступен бесплатно по образовательной лицензии)
2. `sklearn.semi_supervised.LabelPropagation`
3. `sklearn.semi_supervised.LabelSpreading`

In [ ]:
# пример построения графа 

import igraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)
    
    
    
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)